In [ ]:
import sys, numpy as np
import openpyxl
print(sys.executable)      # should point to ...\envs\retinaface_env\python.exe
print(np.__version__)      # should print 1.16.6
print(openpyxl.__version__)# expect >= 3.1.2 (py3.10)

In [ ]:
import os
import csv
import glob
from pathlib import Path
import cv2
import numpy as np
import pandas as pd
import time
import subprocess
import sys

In [ ]:
# Replace with your folder path
image_folder = "C:/Users/Sandip/Desktop/YuNet/Data" 
outIm = "C:/Users/Sandip/Desktop/insightface/detection/retinaface/OutImDir"
outFile = "C:/Users/Sandip/Desktop/insightface/detection/retinaface/OutFileDir"
imrs="C:/Users/Sandip/Desktop/insightface/detection/retinaface/OutImDir/imrs" # resize image save dir

In [ ]:
from insightface.app import FaceAnalysis
import numpy as np
import cv2

class DmlRetinaFaceCompat:
    def __init__(self, det_size=(640, 640)):
        # Use DirectML provider on Windows GPU; CPU fallback stays in list
        self.app = FaceAnalysis(name='buffalo_l',
                                providers=['DmlExecutionProvider', 'CPUExecutionProvider'])
        self.app.prepare(ctx_id=0, det_size=det_size)

    def detect(self, img_bgr, threshold=0.8, scale_div=1.0):
        """
        scale_div = fc. If fc>1, I downscaled by (1/fc) for speed, then map boxes/landmarks back.
        Returns:
          faces: Nx5 [x1,y1,x2,y2,score] in ORIGINAL image coords
          landmarks: Nx5x2 in ORIGINAL image coords (or None)
        """
        # 1) Optional downscale for speed
        if scale_div and float(scale_div) > 1.0:
            h, w = img_bgr.shape[:2]
            new_w = max(1, int(w / float(scale_div)))
            new_h = max(1, int(h / float(scale_div)))
            img_in = cv2.resize(img_bgr, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
            sdiv = float(scale_div)
        else:
            img_in = img_bgr
            sdiv = 1.0

        # 2) Detect on (possibly) downscaled image
        faces_list = self.app.get(img_in)
        if not faces_list:
            return None, None

        # 3) Map results back to original coordinates
        boxes, kps = [], []
        for f in faces_list:
            score = float(f.det_score)
            if score < float(threshold):
                continue
            x1, y1, x2, y2 = f.bbox.astype(float)
            if sdiv != 1.0:
                x1 *= sdiv; y1 *= sdiv; x2 *= sdiv; y2 *= sdiv
                kp = (f.kps.astype(float) * sdiv).astype(np.float32)
            else:
                kp = f.kps.astype(np.float32)
            boxes.append([int(round(x1)), int(round(y1)),
                          int(round(x2)), int(round(y2)), score])
            kps.append(kp)

        if not boxes:
            return None, None
        return np.array(boxes, dtype=np.float32), np.stack(kps, axis=0)

In [ ]:
## Use collect_images is you have problem with os.listdir()
def collect_images(root, exts):
    exts = [e.strip(". ").lower() for e in exts.split(",") if e.strip()]
    paths = []
    for e in exts:
        paths.extend(glob.glob(os.path.join(root, f"**/*.{e}"), recursive=True))
        paths.extend(glob.glob(os.path.join(root, f"*.{e}")))
    return sorted(set(paths))


def ensure_dir(d):
    if d and not os.path.isdir(d):
        os.makedirs(d, exist_ok=True)
        
def file_formatting(filename):
    splt=filename.split('_')
    #print(splt)
    sub_ID=splt[0]
    file=filename
    im_type=splt[1]
    if im_type=="in":
        im_T="indoor"
    else:
        im_T="outdoor"
    device=splt[2]
    if device=='c':
        im_De="camera"
    else:
        im_De="Mobile"
    distance=splt[3]
    category=splt[4]
    return sub_ID, file, im_T, im_De, distance, category   


def draw_and_save(img_bgr, faces, landmarks, out_path):
    if faces is None or len(faces) == 0:
        ensure_dir(os.path.dirname(out_path))
        cv2.imwrite(out_path, img_bgr)
        return
    n = faces.shape[0] if hasattr(faces, "shape") else len(faces)
    for i in range(n):
        x1, y1, x2, y2 = faces[i, 0:4].astype(int)
        score = faces[i, 4]
        cv2.rectangle(img_bgr, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img_bgr, f"{score:.2f}", (x1, max(0, y1 - 5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
        if landmarks is not None:
            lm = landmarks[i].astype(int)
            for (x, y) in lm:
                cv2.circle(img_bgr, (x, y), 2, (255, 0, 0), -1)
    ensure_dir(os.path.dirname(out_path))
    cv2.imwrite(out_path, img_bgr)

In [ ]:
outIm_orig = os.path.join(outIm, 'original')
os.makedirs(outIm_orig, exist_ok=True)

csv_format=pd.DataFrame()
results=pd.DataFrame()
detector = DmlRetinaFaceCompat(det_size=(640,640))
fc=1.0
j=0
files = os.listdir(image_folder)
for filename in files:
    print(filename)
#filename = "63_in_c_10m_spoof1_61_prop_small.jpg"#files[3]
    if filename.endswith((".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG")): # Add other image extensions if needed
        image_path = os.path.join(image_folder, filename)
        img1 = cv2.imread(image_path)
        if img1 is not None:
            input_size = img1.shape  # width x height
        else:
            input_size = 0
            print(f"[WARN] Could not read {image_path}")
            continue
        print(input_size)
        height, width = img1.shape[:2]
        print(height,width)
        
        dsize = (int(width/fc), int(height/fc))
        # Resize the image using cv2.resize if necessary
        img = cv2.resize(img1, dsize)
        print(img.shape)
        im_path=os.path.join(imrs, filename)
        cv2.imwrite(im_path,img)
        sub_ID, file, im_T, im_De, distance, category=file_formatting(filename)
        out_path_Im = os.path.join(outIm, filename)
        try:
            start_time = time.time()
            faces, landmarks = detector.detect(
            img,
            threshold=(float(args.threshold) if 'args' in globals() else 0.8),
            scale_div=1.0
            )
            end_time = time.time()
            
            # Also annotate ORIGINAL image by mapping resized coords back
            H, W = img1.shape[:2]; rs_h, rs_w = img.shape[:2]
            fx, fy = W/float(rs_w), H/float(rs_h)
            faces_orig = None if faces is None else faces. astype(float).copy()
            if faces_orig is not None and ((hasattr(faces_orig,"shape") and faces_orig.shape[0]>0) or (not hasattr(faces_orig,"shape") and len(faces_orig)>0)):
                faces_orig[:, [0,2]] *= fx; faces_orig[:, [1,3]] *= fy
                faces_orig[:, [0,2]] = np.clip(faces_orig[:, [0,2]], 0, W-1)
                faces_orig[:, [1,3]] = np.clip(faces_orig[:, [1,3]], 0, H-1)
                faces_orig = faces_orig.astype(int)
                kps_orig = None
                if landmarks is not None:
                    kps_orig = landmarks.astype(float).copy()
                    kps_orig[:, :, 0] *= fx; kps_orig[:, :, 1] *= fy
                    kps_orig = np.clip(kps_orig, [0,0], [W-1,H-1]).astype(np.int32)
                out_path_orig = os.path.join(outIm_orig, filename)
                draw_and_save(img1.copy(), faces_orig, kps_orig, out_path_orig)
            print('Drawing OK')
            detection_time = round(end_time - start_time, 2)
            print(detection_time)
        except subprocess.CalledProcessError as exc:
            print(f"Command failed with exit code {exc.returncode}")
            print("Standard output:", exc.stdout)
            print("Standard error:", exc.stderr)
        except FileNotFoundError:
            print("Error: Command not found. Ensure it's in your PATH or provide full path.")

        def _count_faces(F):
            if F is None:
                return 0
            try:
                return int(F.shape[0])
            except AttributeError:
                try:
                    return int(len(F))
                except Exception:
                    return 0
        
        # count safely
        n_faces = 0 if faces is None else (faces.shape[0] if hasattr(faces, "shape") else len(faces))
        
        # ---- NO-DETECTION PATH (single row, then continue) ----
        if n_faces == 0:
            detection_status = "No Detection"
            results.loc[j, 0]  = sub_ID
            results.loc[j, 1]  = file
            results.loc[j, 2]  = im_T
            results.loc[j, 3]  = im_De
            results.loc[j, 4]  = distance
            results.loc[j, 5]  = category
            results.loc[j, 6]  = detection_status
            results.loc[j, 7]  = " "                      # box_coords should be empty string, not 0
            results.loc[j, 8]  = detection_time
            results.loc[j, 9]  = str(input_size)
            results.loc[j, 10] = f"{width} X {height}"
            j += 1
            continue   # <- prevents any “detected” rows for this image
        
        # ---- DETECTION PATH (one row per face) ----
        for i in range(n_faces):
            x1, y1, x2, y2 = faces[i, 0:4].astype(int)
            bbox_str = f"[{x1},{y1},{x2},{y2}]"
            results.loc[j, 0]  = sub_ID
            results.loc[j, 1]  = file
            results.loc[j, 2]  = im_T
            results.loc[j, 3]  = im_De
            results.loc[j, 4]  = distance
            results.loc[j, 5]  = category
            results.loc[j, 6]  = "Detected"
            results.loc[j, 7]  = bbox_str
            results.loc[j, 8]  = detection_time
            results.loc[j, 9]  = str(input_size)
            results.loc[j, 10] = f"{width} X {height}"
            j += 1

results.columns=["subject number", "image name", "indoor/outdoor", "device", "distance", "category", "detection", "face box coordinate[xmin,ymin,xmax,ymax]", "detection time (s)", "Original image size", "Face detection size (w*h)"]

os.chdir(outFile)
results.to_excel('RetinaFace.xlsx', index=False)